In [ ]:
import pandas as pd
from datetime import datetime, timedelta

In [11]:
# Load Excel file into a DataFrame
file_path = '/Assignment_Timecard.xlsx'
df = pd.read_excel(file_path)

In [12]:
# Rename columns
df = df.rename(columns={
    'position id': 'Position ID',
    'position status': 'Position Status',
    'time': 'Time',
    'time_out': 'Time Out',
    'timecard': 'Timecard Hours (as Time)',
    'pay cycle': 'Pay Cycle End Date',
    'emp name': 'Employee Name',
    'file number': 'File Number'
})

In [13]:
# Function to convert timecard to timedelta
def convert_to_timedelta(time_str_or_float):
    if pd.isna(time_str_or_float):
        return timedelta(hours=0, minutes=0)

    if isinstance(time_str_or_float, float):
        hours = int(time_str_or_float)
        minutes = int((time_str_or_float - hours) * 60)
    elif isinstance(time_str_or_float, str):
        hours, minutes = map(int, time_str_or_float.split(':'))
    else:
        raise ValueError("Invalid time format")

    return timedelta(hours=hours, minutes=minutes)

In [14]:
# Convert time columns to datetime objects
df['Time'] = pd.to_datetime(df['Time'])
df['Time Out'] = pd.to_datetime(df['Time Out'])
df['Pay Cycle End Date'] = pd.to_datetime(df['Pay Cycle End Date'])

In [15]:
# Convert timecard to timedelta
df['Timecard Hours (as Time)'] = df['Timecard Hours (as Time)'].apply(convert_to_timedelta)

In [19]:
# Filter employees who worked for 7 consecutive days
consecutive_days = 7
consecutive_days_mask = df.groupby('Employee Name')['Time'].diff().dt.days.eq(1)
consecutive_days_employees = df[consecutive_days_mask].drop_duplicates(subset='Employee Name')
print("Employees who worked for 7 consecutive days:")
print(consecutive_days_employees[['Employee Name', 'Position ID']])

Employees who worked for 7 consecutive days:
                    Employee Name Position ID
36                Sparks, KeWWeMh   WFS000101
75                  Arias, FeXipe   WFS000170
106                GarWes, JAsMiW   WFS000181
126                 FeXix, XeEWeX   WFS000183
179                    Xee, XaCar   WFS000200
...                           ...         ...
1388   De XEs SaWMEs, JEseph XAis   WFS000582
1399        ViXXa, AXfredE AieXXE   WFS000583
1412     ArCeWMa FXEres, JEWaMhaW   WFS000584
1434  RaCEs HerWaWdez, AdieX ECar   WFS000585
1453       XariEs, Cesar GiEvaWWi   WFS000587

[68 rows x 2 columns]


In [20]:
# Filter employees with less than 10 hours between shifts
min_hours_between_shifts = 1
max_hours_between_shifts = 10
less_than_10_hours_mask = (df['Time'].diff() < timedelta(hours=max_hours_between_shifts)) & (df['Time'].diff() > timedelta(hours=min_hours_between_shifts))
less_than_10_hours_employees = df[less_than_10_hours_mask].drop_duplicates(subset='Employee Name')
print("\nEmployees with less than 10 hours between shifts but greater than 1 hour:")
print(less_than_10_hours_employees[['Employee Name', 'Position ID']])


Employees with less than 10 hours between shifts but greater than 1 hour:
                    Employee Name Position ID
2                 REsaXiaWE, XAis   WFS000065
21                Sparks, KeWWeMh   WFS000101
41            GaXCes, EXias XEpez   WFS000127
53               CaMaXaWE, CeghaW   WFS000153
72                  Arias, FeXipe   WFS000170
...                           ...         ...
1409     ArCeWMa FXEres, JEWaMhaW   WFS000584
1427  RaCEs HerWaWdez, AdieX ECar   WFS000585
1444       XariEs, Cesar GiEvaWWi   WFS000587
1464     AGeX, WaMhaW REGerM PeWa   WFS000588
1474              WgAyeW, RayCEWd   WFS000589

[96 rows x 2 columns]


In [22]:
# Filter employees who worked more than 14 hours in a single shift
max_hours_single_shift = 14
more_than_14_hours_mask = (df['Time Out'] - df['Time']) > timedelta(hours=max_hours_single_shift)
more_than_14_hours_employees = df[more_than_14_hours_mask].drop_duplicates(subset='Employee Name')
print("\nEmployees who worked for more than 14 hours in a single shift:")
print(more_than_14_hours_employees[['Employee Name', 'Position ID']])


Employees who worked for more than 14 hours in a single shift:
                        Employee Name Position ID
1036  DeXgadiXXE REdarMe, ChrisMiaW S   WFS000523


In [25]:
output_file_path = '/output.txt'
with open(output_file_path, 'w') as output_file:
    output_file.write("Employees who worked for 7 consecutive days:\n")
    output_file.write(consecutive_days_employees[['Employee Name', 'Position ID']].to_string(index=False))
    output_file.write("\n\nEmployees with less than 10 hours between shifts but greater than 1 hour:\n")
    output_file.write(less_than_10_hours_employees[['Employee Name', 'Position ID']].to_string(index=False))
    output_file.write("\n\nEmployees who worked for more than 14 hours in a single shift:\n")
    output_file.write(more_than_14_hours_employees[['Employee Name', 'Position ID']].to_string(index=False))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')